Based on [this guide](https://langfuse.com/docs/integrations/openai/python/get-started)

In [ ]:
%pip install langfuse openai --upgrade

In [ ]:
# instead of: import openai
from langfuse.openai import openai

In [ ]:
# For debugging, checks the SDK connection with the server. Do not use in production as it adds latency.
openai.langfuse_auth_check()

In [ ]:
completion = openai.chat.completions.create(
    name="test-chat",
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system",
            "content": "You are a very accurate calculator. You output only the result of the calculation.",
        },
        {"role": "user", "content": "1 + 1 = "},
    ],
    temperature=0,
    metadata={"someMetadataKey": "someValue"},
)

In [ ]:
from langfuse.openai import openai
from langfuse.decorators import observe


@observe()  # decorator to automatically create trace and nest generations
def main(country: str, user_id: str, **kwargs) -> str:
    # nested generation 1: use openai to get capital of country
    capital = (
        openai.chat.completions.create(
            name="geography-teacher",
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "You are a Geography teacher helping students learn the capitals of countries. Output only the capital when being asked.",
                },
                {"role": "user", "content": country},
            ],
            temperature=0,
        )
        .choices[0]
        .message.content
    )

    # nested generation 2: use openai to write poem on capital
    poem = (
        openai.chat.completions.create(
            name="poet",
            model="gpt-3.5-turbo",
            messages=[
                {
                    "role": "system",
                    "content": "You are a poet. Create a poem about a city.",
                },
                {"role": "user", "content": capital},
            ],
            temperature=1,
            max_tokens=200,
        )
        .choices[0]
        .message.content
    )

    return poem


# run main function and let Langfuse decorator do the rest
print(main("Bulgaria", "admin"))